# 11/10/20
Testing GPyOpt functionality with their manual examples and functions from https://en.wikipedia.org/wiki/Test_functions_for_optimization. 1D and 2D straightforward with visuals. Tried with higher dimensional problems (n=10,15) - sphere function was fine, but 